In [ ]:
import numpy as np
from clawpack import riemann
from  clawpack.visclaw import ianimate

In [ ]:
def q_src(solver, state,  dt):
    lamda = state.problem_data['lambda']
    dt2 = dt/2.
    q = state.q
    qstar = np.empty(q.shape)
    qstar[0,:] = q[0,:] - dt2*lamda*q[0,:]
    q[0,:] = q[0,:] - dt*lamda*q[0,:]

def setup(maxslope,  lamda=1., outdir='./_output',solver_type='classic'):

    from clawpack import pyclaw

    riemann_solver = riemann.burgers_1D

    if solver_type=='sharpclaw':
        solver = pyclaw.SharpClawSolver1D(riemann_solver)
    else:
        solver = pyclaw.ClawSolver1D(riemann_solver)
        solver.limiters = pyclaw.limiters.tvd.vanleer
        solver.step_source = q_src
        solver.source_split = 1.

    solver.bc_lower[0] = pyclaw.BC.periodic
    solver.bc_upper[0] = pyclaw.BC.periodic

    x = pyclaw.Dimension(0.0,2*np.pi,500,name='x')
    domain = pyclaw.Domain(x)
    num_eqn = 1
    state = pyclaw.State(domain,num_eqn)

    xc = state.grid.x.centers
    state.q[0,:] = maxslope*np.sin(xc) + 1.
    state.problem_data['efix']=True
    state.problem_data['lambda'] = lamda
    
    claw = pyclaw.Controller()
    claw.tfinal = 10.
    claw.solution = pyclaw.Solution(state,domain)
    claw.solver = solver
    claw.outdir = outdir
    #claw.setplot = setplot
    claw.keep_copy = True
    claw.num_output_times = 50

    return claw

In [ ]:
claw = setup(maxslope=0.1)
claw.verbosity = 0
claw.run()

In [ ]:
ianimate.ianimate(claw)